# LASSO - coordinate descent algorithm

* Function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1473052407.log


This non-commercial license of GraphLab Create for academic use is assigned to aguha2@jhu.edu and will expire on February 11, 2017.


In [3]:
import numpy as np # note this allows us to refer to numpy as np instead 

In [4]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]  
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe['price'] 
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

In [5]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix,weights)
    return(predictions)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [6]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[  3.   5.   8.]
 [  4.  12.  15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [7]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms

[  5.  13.  17.]


To normalize, apply element-wise division:

In [8]:
print X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [9]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized_features= feature_matrix/norms
    return (normalized_features, norms)

## Testing the code

To test the function, run the following:

In [12]:
feature_mat= np.array([[3.,6.,9.],[4.,8.,12.]])
features, norms = normalize_features(feature_mat)
print features
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print norms
# should print
# [5.  10.  15.]

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [22]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
output

array([ 221900.,  538000.,  180000., ...,  402101.,  400000.,  325000.])

Don't forget to normalize features:

In [23]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)
simple_feature_matrix

array([[ 0.00680209,  0.00353021,  0.00583571],
       [ 0.00680209,  0.00768869,  0.00583571],
       [ 0.00680209,  0.00230361,  0.00389048],
       ..., 
       [ 0.00680209,  0.00305154,  0.00389048],
       [ 0.00680209,  0.00478673,  0.00583571],
       [ 0.00680209,  0.00305154,  0.00389048]])

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [24]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [25]:
prediction = predict_output(simple_feature_matrix, weights)
prediction

array([ 0.02675867,  0.04339256,  0.01990703, ...,  0.02289873,
        0.03178473,  0.02289873])

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [26]:
def calculate_ro(num_features, feature_matrix,output,prediction, weights):
    for i in range(num_features):
        ro_i = (feature_matrix[:,i]*(output - prediction+
        weights[i]*feature_matrix[:,i])).sum()
        #ro_arr.append(ro_i)
        print ro_i
        print i
    return 0

ro = calculate_ro(3, simple_feature_matrix,output,prediction, weights)
ro


#calculate_ro(3, simple_feature_matrix,output,prediction, weights)


79400300.0349
0
87939470.773
1
80966698.676
2


0

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [27]:
def lasso_coordinate_descent_step(num_features, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    #z_i= (feature_matrix*feature_matrix).sum()

    for i in range(num_features+1):
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
        ro_i = (feature_matrix[:,i]*(output - prediction+weights[i]*feature_matrix[:,i])).sum()
        if i == 0: # intercept -- do not regularize
            new_weight_i = ro_i 
        elif ro_i < -l1_penalty/2.:
            new_weight_i = (ro_i +(l1_penalty/2))
        elif ro_i > l1_penalty/2.:
            new_weight_i = (ro_i -(l1_penalty/2))
        else:
            new_weight_i = 0.
    return new_weight_i

To test the function, run the following cell:

In [28]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array(([3./math.sqrt(13),1./math.sqrt(10)],
                                                 [2./math.sqrt(13),3./math.sqrt(10)])), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1)
print "finally, watch for paranthesis errors.."

0.425558846691
finally, watch for paranthesis errors..


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. Loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights



In [29]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, 
                                      l1_penalty, tolerance):
    condition = True
    max_change=0
    while (condition == True):  
        max_change=0
        for i in range (len(initial_weights)):
            #max_change=0
            old_weight_i= initial_weights[i]
            initial_weights[i] =lasso_coordinate_descent_step(i,
                                                      feature_matrix, output, 
                                                      initial_weights, l1_penalty)
            coordinate_change = abs(old_weight_i - initial_weights[i])
            if coordinate_change>max_change:
                max_change = coordinate_change
                print max_change
        if (coordinate_change < tolerance):
            condition = False
    return initial_weights        

        

Using the following parameters, learn the weights on the sales dataset. 

In [30]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`

In [31]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [32]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

weights

79400304.6581
9138168.3665
8194809.40937


array([ 62067326.88218397,  26161208.08576558,         0.        ])

In [34]:
prediction = predict_output(normalized_simple_feature_matrix, weights)

rss_errors = (sales['price'] - prediction)
        # Then square and add them up
rss_error_sq= rss_errors*rss_errors
        #RSS = RSS_ + str(degree)
RSS=rss_error_sq.sum()
#print RSS

#rss_errors

# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [36]:
train_data,test_data = sales.random_split(.8,seed=0)

Let us consider the following set of features.

In [37]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

In [38]:
(traindata_feature_matrix, output_train) = get_numpy_data(train_data, all_features, 'price')

(trainingdata_normalized, norms) = normalize_features(traindata_feature_matrix)



First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [39]:
l1_penalty=1e7
initial_weights = np.zeros(14)
tolerance = 1.0

weights1e7 = lasso_cyclical_coordinate_descent(trainingdata_normalized, output_train,
                                            initial_weights, l1_penalty, tolerance)


71114625.7528


In [33]:
weights1e7

array([ 71114625.75280938,         0.        ,   3743972.43191673,
         5271064.34696085,         0.        ,         0.        ,
         7173100.28480826,   7025132.06642577,  -5530804.65691784,
               0.        ,    394565.58439511,   2242690.39485069,
        -2160960.47385677,         0.        ])

## Some features had non-zero weight above this case

Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [41]:
l1_penalty=1e8
initial_weights = np.zeros(14)
tolerance = 1.0
weights1e8 = lasso_cyclical_coordinate_descent(trainingdata_normalized, output_train,
                                            initial_weights, l1_penalty, tolerance)
weights1e8


71114625.7528


array([ 71114625.75280938,         0.        ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ])

In [44]:
l1_penalty=1e4
initial_weights = np.zeros(14)
tolerance = 5e5
weights1e4 = lasso_cyclical_coordinate_descent(trainingdata_normalized, output_train,
                                            initial_weights, l1_penalty, tolerance)
weights1e4

71114625.7528
656582.579069
2468988.04934
3584457.8054
4916244.61106
4931522.9822
5297036.06628
8464015.94738


array([ 70458043.17373988,   6425368.15103951,   8547900.27630541,
         8914874.6783465 ,  -1450214.98167019, -13686427.97272057,
         7599969.4022666 ,  12239550.5873264 , -12644466.52258853,
         6608416.91505529,  11849744.63068959,   3426663.58888805,
       -20051994.77410842,   3192423.86076874])

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

In [45]:
(testdata_feature_matrix, output_train) = get_numpy_data(test_data, all_features, 'price')

(testdata_normalized, norms_test) = normalize_features(testdata_feature_matrix)

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [46]:
#features, norms_test = normalize_features(all_features)


normalized_weights1e7=weights1e7/norms_test
print normalized_weights1e7[3]
normalized_weights1e7

35.7407807162


array([  1.09355363e+06,   0.00000000e+00,   2.56192827e+04,
         3.57407807e+01,   0.00000000e+00,   0.00000000e+00,
         1.30962294e+06,   1.36313999e+05,  -2.45363212e+04,
         0.00000000e+00,   3.07280586e+00,   6.60571227e+01,
        -1.68626217e+01,   0.00000000e+00])